In [ ]:
from serve import H2OxHandler
from ts.context import Context
import xarray as xr
import json
    
%load_ext autoreload
%autoreload 2

In [ ]:
ctx = Context(
    model_name='kaveri', 
    model_dir='/home/lucas/h2ox-ai/models/', 
    manifest = {'model': 
                {
                    'serializedFile': 'kaveri.pt',
                }
               }, 
    batch_size=8, 
    gpu='0', 
    mms_version=0.1, 
    limit_max_image_pixels=True
)

In [ ]:
inst = H2OxHandler()
inst.initialize(ctx)

In [ ]:
import os, pickle
import pandas as pd
from datetime import datetime, timedelta
import numpy as np

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import torch

In [ ]:
ds = xr.open_dataset('./../data/data_kaveri.nc')

In [ ]:
preds = xr.open_dataset('./../experiments_2/sacred/147/preds.nc')

In [ ]:
dl_data = pickle.load(open('./../data/ds_data.pkl','rb'))
dl_meta = pickle.load(open('./../data/ds_meta.pkl','rb'))

In [ ]:
sample_data = json.load(open('./../data/kaveri_sample_2020_10_01.json','r'))

In [ ]:
[idx for idx in range(len(dl_meta)) if dl_meta[idx]['date']==np.datetime64(datetime(2020,10,1).isoformat())]

In [ ]:
trn_sample = dl_data[3836]

In [ ]:
inps = inst.preprocess(sample_data)

In [ ]:
y_hat = inst.inference(inps)

In [ ]:
results = inst.postprocess(y_hat, inps)

In [ ]:
results['2020-10-01']['lower']

In [ ]:
sdt = datetime(2020,10,1)

In [ ]:
hist_data = {site:sample_data['2020-10-01'][site]['y']['targets_WATER_VOLUME'] for site in sample_data['2020-10-01'].keys()}

In [ ]:
hist_df = pd.DataFrame(hist_data, index = pd.date_range(sdt-timedelta(days=90),sdt))

In [ ]:
dfs = {
    kk: pd.DataFrame(
        results['2020-10-01'][kk].cpu().detach().numpy(), 
        index=pd.date_range(sdt,sdt+timedelta(days=89)),
        columns=hist_data.keys()
    )
    
    for kk in ['lower','upper','mean']}

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(16,8))
hist_df.plot(ax=ax)
for kk in dfs.keys():
    dfs[kk].plot(ax=ax)